In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction-parquet-files/test_file_23.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_72.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_61.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_55.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_71.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_53.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_30.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_20.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_80.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_85.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_36.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_50.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_97.pqt
/kaggle/input/amex-default-prediction-parquet-files/train_file_28.pqt
/kaggle/input/amex-default-predic

## Please refer to [this](https://www.kaggle.com/code/sravanneeli/convert-train-and-test-multiple-parquet-files) notebook for creating train and test parquet files

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 13.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ba0edf2990ea619324df733da2e7471d7ef9f289dd6b4f2190afdae8ea2656f4
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('amex-test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/01 08:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
train_df = spark.read.parquet('../input/amex-default-prediction-parquet-files/train*.pqt')

In [5]:
train_df.printSchema()

root
 |-- customer_ID: binary (nullable = true)
 |-- S_2: timestamp (nullable = true)
 |-- P_2: float (nullable = true)
 |-- D_39: float (nullable = true)
 |-- B_1: float (nullable = true)
 |-- B_2: float (nullable = true)
 |-- R_1: float (nullable = true)
 |-- S_3: float (nullable = true)
 |-- D_41: float (nullable = true)
 |-- B_3: float (nullable = true)
 |-- D_42: float (nullable = true)
 |-- D_43: float (nullable = true)
 |-- D_44: float (nullable = true)
 |-- B_4: float (nullable = true)
 |-- D_45: float (nullable = true)
 |-- B_5: float (nullable = true)
 |-- R_2: float (nullable = true)
 |-- D_46: float (nullable = true)
 |-- D_47: float (nullable = true)
 |-- D_48: float (nullable = true)
 |-- D_49: float (nullable = true)
 |-- B_6: float (nullable = true)
 |-- B_7: float (nullable = true)
 |-- B_8: float (nullable = true)
 |-- D_50: float (nullable = true)
 |-- D_51: float (nullable = true)
 |-- B_9: float (nullable = true)
 |-- R_3: float (nullable = true)
 |-- D_52: float (

In [6]:
from pyspark.sql.types import StringType, StructType
import pyspark.sql.functions as func 

In [7]:
train_df = train_df.withColumn("customer_ID",train_df["customer_ID"].cast(StringType()))

In [8]:
print(f"Total number of rows: {train_df.count()} and Tota number of cols: {len(train_df.columns)}")

Total number of rows: 5531451 and Tota number of cols: 190


In [9]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [10]:
num_cols = []
for col in train_df.columns[2:]:
    if col not in cat_cols:
        num_cols.append(col)


## Group By for each Individual Customer ID

## The usage of pyspark
* I observed in other notebook they where using feather data which was created by somebody but there was no proper source where that has been created.
* So I though and first create parquet files from csv files which I attached above.
* Once of we have parquet files we can now load `train` and `test` data using pyspark which will do aggregation with parallel manner with very less concumption RAM.
* Order each pyspark dataframe with `customer_ID` and at the end convert them to pandas DataFrame and just concat them horizontally because all are sorted.

In [11]:
%%time
count_exprs = {col: 'count' for col in cat_cols}
cat_count_df = train_df.groupby('customer_ID').agg(count_exprs).orderBy('customer_ID', ascending=True).toPandas()

CPU times: user 5.36 s, sys: 353 ms, total: 5.71 s
Wall time: 20.1 s


In [12]:
for col in cat_count_df.columns[1:]:
    cat_count_df[col] = cat_count_df[col].astype('int8')

In [13]:
cat_count_df.shape

(458913, 12)

In [14]:
gc.collect()

79

In [15]:
%%time
last_exprs = {col: 'last' for col in cat_cols}
cat_last_df = train_df.groupby('customer_ID').agg(last_exprs).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1)

CPU times: user 4.4 s, sys: 324 ms, total: 4.73 s
Wall time: 17.3 s


In [16]:
for col in cat_last_df:
    if cat_last_df[col].dtype == "float32":
        cat_last_df[col] = cat_last_df[col].astype('float16')

In [17]:
cat_last_df.shape

(458913, 11)

In [18]:
gc.collect()

72

In [19]:
def grp_unique_count(train_df):
    agg_df = []
    for col in cat_cols:
        agg_df.append(train_df.groupby('customer_ID').agg(func.expr(f'count(distinct {col})').alias(f'nunique({col})')).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1))
    final_df = pd.concat(agg_df, axis=1).astype('int8')
    gc.collect()
    return final_df

In [20]:
%%time
cat_nc_df = grp_unique_count(train_df)

CPU times: user 32.7 s, sys: 319 ms, total: 33 s
Wall time: 1min 27s


In [21]:
cat_cols_df = pd.concat([cat_count_df, cat_last_df, cat_nc_df], axis=1)

In [22]:
cat_cols_df.to_pickle('cat_cols_df.pkl', compression='gzip')

In [23]:
del(cat_count_df)
del(cat_last_df)
del(cat_nc_df)
del(cat_cols_df)

In [24]:
gc.collect()

84

In [25]:
def grp_num_cols(train_df):
    def agg_num(agg_func):
        agg_df = []
        for i in range(0, len(num_cols), 20):
            exprs = {col: agg_func for col in num_cols[i:i+20]}
            agg_df.append(train_df.groupBy('customer_ID').agg(exprs).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1))
        final_df = pd.concat(agg_df, axis=1).astype('float16')
        gc.collect()
        return final_df
    num_mean_df = agg_num("mean")
    num_std_df = agg_num("std")
    num_min_df = agg_num("min")
    num_max_df = agg_num("max")
    final_df = pd.concat([num_mean_df, num_std_df, num_min_df, num_max_df], axis=1)
    return final_df

In [26]:
%%time
num_cols_df = grp_num_cols(train_df)

22/06/01 08:25:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 3min 38s, sys: 24.2 s, total: 4min 3s
Wall time: 10min 11s


In [27]:
num_cols_df.to_pickle('num_cols_df.pkl', compression='gzip')

In [28]:
del(num_cols_df)

In [29]:
cat_cols_df = pd.read_pickle('./cat_cols_df.pkl', compression='gzip')
num_cols_df = pd.read_pickle('./num_cols_df.pkl', compression='gzip')
final_train_df = pd.concat([cat_cols_df, num_cols_df], axis=1)

In [30]:
del(cat_cols_df)
del(num_cols_df)
gc.collect()

21

In [31]:
final_train_df.to_pickle('train_agg.pkl', compression='gzip')